In [1]:
import pandas as pd
import git
import mlflow.keras
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics import *
from tensorflow.keras.layers import *
from keras.models import *
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models.wrappers import FastText


Using TensorFlow backend.
/home/joshib/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/home/joshib/anaconda3/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

In [3]:
path = '/home/joshib/cs_data/citation-bio-labelled-data-2020-06-08~15:13:57.766608/nlp/exps/output/2020-06-08~15:13:57.766608'
#data_df = pd.read_csv(path+'/data-2020-06-08~15:14:01.185827.csv')

## LSTM with random embedding model

In [4]:
train_data_path = '/home/joshib/cs_data/citation-bio-labelled-data-2020-06-09~06:30:11.273245/nlp/exps/output/2020-06-09~06:30:11.273245'

sentences = []
sent_tags = []
len_arr = []
for fpath in os.listdir(train_data_path):
    if fpath not in ['data-gen-config.json', 'data_generation_stats.csv']:
        fpath = os.path.join(train_data_path, fpath)
        #print(fpath)
        df = pd.read_csv(fpath, index_col=0)
        df.fillna("\n", axis=1, inplace=True)
        len_arr.append(df.shape[0])
        sentences.append(" ".join(df.x))
        sent_tags.append(" ".join(df.y))

In [5]:
import numpy
print("Mean: %d"%numpy.mean(len_arr))
print("Max: %d"%numpy.max(len_arr))

Mean: 1321
Max: 2223


In [6]:
%%time
maxlen = 2223
X = [[w for w in s.split()] for s in sentences]
new_X = []
for seq in X:
    new_seq = []
    for i in range(maxlen):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("PADword")
    new_X.append(new_seq)


CPU times: user 323 ms, sys: 39.9 ms, total: 363 ms
Wall time: 362 ms


In [7]:
embed_size = 300 # how big is each word vector
max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
#maxlen = 1000 # max number of words in a question to use

In [8]:
%%time
from keras.preprocessing.sequence import pad_sequences
tags = df.y.unique()
tags2index = {t:i for i,t in enumerate(tags)}
y = [[tags2index[w] for w in s.split()] for s in sent_tags]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=tags2index["I-CIT"])

CPU times: user 285 ms, sys: 2.97 ms, total: 288 ms
Wall time: 288 ms


In [9]:
X_train, X_test, y_train, y_test = train_test_split(new_X, y, test_size=0.1, random_state=42)

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

print("Padding sequences for uniform dimensions")
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

Padding sequences for uniform dimensions


## Keras with pre-trained fast text embedding

In [11]:
%%time
wiki_model = FastText.load_fasttext_format('/newvolume/cc.en.300.bin')

CPU times: user 2min 11s, sys: 4.97 s, total: 2min 16s
Wall time: 2min 33s


In [12]:
%%time
def load_embedding_matrix(embeddings, nb_words, word_index, embed_dim):  # load embeddings
    embeddings_index = {}
    for word in embeddings.wv.vocab:
        embeddings_index[word] = embeddings[word]
    print('Found %s word vectors.' % len(embeddings_index))

    words_not_found = []
    embedding_matrix = np.zeros((nb_words, embed_dim))
    for word, i in word_index.items():
        if i >= nb_words:
            continue
        try:
            embedding_vector = embeddings[word]
            embedding_matrix[i] = embedding_vector
        except:
            words_not_found.append(word)
    print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix


embedding_matrix = load_embedding_matrix(wiki_model, max_features, tokenizer.word_index, 300)

Found 2000000 word vectors.
number of null word embeddings: 15478
CPU times: user 8.14 s, sys: 256 ms, total: 8.39 s
Wall time: 8.39 s


In [13]:
%%time
model = tf.keras.models.Sequential()
model.add(Embedding(max_features, 300, weights=[embedding_matrix], trainable=True, input_length= maxlen))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
model.add(TimeDistributed(Dense(1, activation="sigmoid")))
# model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.005,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

/home/joshib/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/keras/layers/recurrent.py:808: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if (isinstance(inputs, collections.Sequence)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2223, 300)         30000000  
_________________________________________________________________
bidirectional (Bidirectional (None, 2223, 1024)        3330048   
_________________________________________________________________
time_distributed (TimeDistri (None, 2223, 1)           1025      
Total params: 33,331,073
Trainable params: 33,331,073
Non-trainable params: 0
_________________________________________________________________
CPU times: user 2.56 s, sys: 732 ms, total: 3.29 s
Wall time: 5.16 s


/home/joshib/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:720: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):


In [17]:
model.fit(X_train, y_train, verbose=1, epochs=20, validation_data=(X_test, y_test))

Train on 400 samples, validate on 45 samples
Epoch 1/20
400/400 [==============================] - 18s 44ms/sample - loss: 0.0967 - accuracy: 0.9806 - val_loss: 0.0958 - val_accuracy: 0.9801
Epoch 2/20
400/400 [==============================] - 17s 43ms/sample - loss: 0.0949 - accuracy: 0.9807 - val_loss: 0.0990 - val_accuracy: 0.9803
Epoch 3/20
400/400 [==============================] - 17s 43ms/sample - loss: 0.0937 - accuracy: 0.9806 - val_loss: 0.0954 - val_accuracy: 0.9798
Epoch 4/20
400/400 [==============================] - 17s 43ms/sample - loss: 0.0928 - accuracy: 0.9806 - val_loss: 0.0976 - val_accuracy: 0.9799
Epoch 5/20
400/400 [==============================] - 17s 43ms/sample - loss: 0.0917 - accuracy: 0.9806 - val_loss: 0.0958 - val_accuracy: 0.9798
Epoch 6/20
400/400 [==============================] - 17s 43ms/sample - loss: 0.0905 - accuracy: 0.9806 - val_loss: 0.0973 - val_accuracy: 0.9799
Epoch 7/20
400/400 [==============================] - 17s 43ms/sample - loss: 0

### Keras with random embedding

In [15]:
%%time
model = tf.keras.models.Sequential()
model.add(Embedding(max_features, embed_size, input_length=maxlen))
model.add(Bidirectional(LSTM(512, return_sequences=True)))
#model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(TimeDistributed(Dense(1, activation="sigmoid")))
# # model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=0.005,
        beta_1=0.0,
        beta_2=0.0,
        epsilon=1e-05,
        amsgrad=False,
    ),
    loss='binary_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2223, 300)         30000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 2223, 1024)        3330048   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 2223, 1)           1025      
Total params: 33,331,073
Trainable params: 33,331,073
Non-trainable params: 0
_________________________________________________________________
CPU times: user 1.82 s, sys: 309 ms, total: 2.13 s
Wall time: 985 ms


In [16]:
model.fit(X_train, y_train, verbose=1, epochs=5, validation_data=(X_test, y_test))

Train on 400 samples, validate on 45 samples
Epoch 1/5
400/400 [==============================] - 21s 53ms/sample - loss: 0.2213 - accuracy: 0.9592 - val_loss: 0.1568 - val_accuracy: 0.9803
Epoch 2/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.1173 - accuracy: 0.9802 - val_loss: 0.0976 - val_accuracy: 0.9803
Epoch 3/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.1049 - accuracy: 0.9802 - val_loss: 0.1092 - val_accuracy: 0.9803
Epoch 4/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.1015 - accuracy: 0.9802 - val_loss: 0.0960 - val_accuracy: 0.9803
Epoch 5/5
400/400 [==============================] - 17s 43ms/sample - loss: 0.0984 - accuracy: 0.9805 - val_loss: 0.1015 - val_accuracy: 0.9803
